In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
import seaborn as sns
from matplotlib.dates import  DateFormatter
%matplotlib inline
from ebmdatalab import bq
from ebmdatalab import charts
from ebmdatalab import maps
import datetime

find consecutive at least 3 month price concessions

In [3]:
sql = """
SELECT
    ncso.date AS month, --month
    ncso.drug AS name,  -- drug name
    vmpp.bnf_code AS bnf_code, --BNF code from VMPP table,
    vmpp.id AS vmpp_code, 
    ncso.price_pence AS pc_price_pence, --price concession cost per pack
    dt.price_pence AS dt_price_pence, --Drug Tariff cost per pack
    qtyval, --VMPP pack size
    (ncso.price_pence - dt.price_pence)/qtyval AS increased_ppu --difference between concession and usual Drug Tariff price
  FROM
    ebmdatalab.dmd.ncsoconcession AS ncso --concession table
  INNER JOIN
    dmd.vmpp_full AS vmpp --VMPP table
  ON
    ncso.vmpp = vmpp.id
  INNER JOIN
    dmd.tariffprice AS dt -- Drug Tariff table
  ON
    ncso.vmpp = dt.vmpp
    AND ncso.date = dt.date
  QUALIFY ROW_NUMBER() OVER (PARTITION BY ncso.date, vmpp.bnf_code ORDER BY (ncso.price_pence - dt.price_pence)/qtyval DESC) = 1 -- for each bnf_code and pack size, calculates PPU difference and ranks in order. Takes the top value, therefore only keeping the highest impact pack size, and thereby removes duplicates for pack size
  ORDER BY
    ncso.date,
    vmpp.bnf_code
"""
exportfile = os.path.join("..","data","pc_df.csv")
pc_df = bq.cached_read(sql, csv_path=exportfile, use_cache=True)
pc_df['month'] = pc_df['month'].astype('datetime64[ns]')

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=725825577420-unm2gnkiprugilg743tkbig250f4sfsj.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&state=RZejnwQ5lyqcnmpe139YiNeVhelMb8&prompt=consent&access_type=offline


Enter the authorization code:  4/1ARtbsJpqM5yZLpUWl7IPjaGmK3cxtju7tmMJMcn8w-KGF1S3bVlMNCy4hBc


Downloading: 100%|██████████| 4656/4656 [00:00<00:00, 7804.54rows/s]


In [8]:
pc_df.head()

,month,name,bnf_code,vmpp_code,pc_price_pence,dt_price_pence,qtyval,increased_ppu
0,2014-08-01,Amiloride 5mg tablets,0202030C0AAACAC,1191111000001100,1950,806,28,40.857142857
1,2014-08-01,Co-amilofruse 5/40 tablets,0202040B0AAAAAA,1245011000001108,529,104,28,15.178571429
2,2014-08-01,Co-amilofruse 2.5/20 tablets,0202040B0AAABAB,1122311000001100,432,122,28,11.071428571
3,2014-08-01,Co-Tenidone 50/12.5mg tablets,020400040AAAAAA,1191711000001104,400,109,28,10.392857143
4,2014-08-01,Co-Tenidone 100/25mg tablets,020400040AAABAB,1132711000001103,400,123,28,9.892857143
